In [2]:
from glob import glob
from tqdm import tqdm
#from rdkit import Chem
import numpy as np
import pandas as pd
import re
import shutil
import os

In [3]:
train = pd.read_csv("data/train_set.ReorgE.csv")
train.columns = ["index", "SMILES", "Reorg_g", "Reorg_ex"]

train.head()

,index,SMILES,Reorg_g,Reorg_ex
0,train_0,CC[C@H]1CCCCN1C(=O)[C@@H](C)OC(=O)c1c(C)oc(-n2...,0.631486,0.535060
1,train_1,O[C@@H](CNC1CC1)CN1CCc2sccc2C1,0.825901,1.116781
2,train_2,N#CCCNC(=O)[C@@]1(O)CCSC1,1.463943,0.964848
3,train_3,COC[C@H]1CN(c2ccc(OCC[C@@H](C)O)cc2)C(=O)O1,0.166669,0.161458
4,train_4,N#Cc1c(-c2ccccc2OCC(N)=O)[nH]c(C(N)=O)c1N,0.313820,0.338862


In [5]:
if os.path.isdir("ex_file"):
    pass
else:
    os.mkdir("ex_file")

if os.path.isdir("g_file"):
    pass
else:
    os.mkdir("g_file")

train_mol = sorted(glob("data/mol_files/train_set/*.mol"))

for i in tqdm(train_mol):
    result = []
    tmp = open(i, 'r').read().split("\n")
    for j in tmp:
        tmp_ = re.sub(' +', ' ', j.lstrip()).split(' ')
        if len(tmp_) > 11:
            result.append(tmp_)
            
    file_name = i.split('\\')[-1].split('.')[0]
    
    if "ex" in file_name:
        pd.DataFrame(result).to_csv(f"ex_file/{file_name}" + ".csv", index = False)
    elif "g" in file_name:
        pd.DataFrame(result).to_csv(f"g_file/{file_name}" + ".csv", index = False)

 78%|███████▊  | 28297/36314 [02:03<00:35, 228.95it/s]


KeyboardInterrupt: 

In [ ]:
max_len = 0
error = []
g_data, ex_data = [], []
mol = []

for i in tqdm(train[train.columns[0]]):
    tmp_g = pd.read_csv(f"g_file/{i}" + '_g.csv')

    if len(tmp_g) >= max_len:
        max_len = len(tmp_g)
        
    for j in tmp_g["3"]:
        mol.append(j)

In [ ]:
mol_list = list(set(mol))
g_data, ex_data = [], []

def get_mol(data):
    if data in mol_list:
        return mol_list.index(data) + 1
    
    else:
        return -1

for i in tqdm(train[train.columns[0]]):
    tmp_g = pd.read_csv(f"g_file/{i}" + '_g.csv').iloc[:, :4]
    tmp_ex = pd.read_csv(f"ex_file/{i}" +'_ex.csv').iloc[:, :4]
    
    tmp_g["3"] = tmp_g["3"].apply(lambda x : get_mol(x))
    tmp_ex["3"] = tmp_ex["3"].apply(lambda x : get_mol(x))
    
    tmp_g = np.array(tmp_g)
    tmp_ex = np.array(tmp_ex)
    
    if max_len != len(tmp_g):
        tmp_g = np.concatenate((tmp_g, np.array([[0] * 4] * (max_len-tmp_g.shape[0]))))
        tmp_ex = np.concatenate((tmp_ex, np.array([[0] * 4] * (max_len - tmp_ex.shape[0]))))
    elif max_len == len(tmp_g):
        tmp_g = tmp_g
        tmp_ex = tmp_ex
    
    g_data.append(tmp_g)
    ex_data.append(tmp_ex)

In [ ]:
g_data = np.array(g_data).reshape(len(g_data), -1)
ex_data = np.array(ex_data).reshape(len(ex_data), -1)

train_x = np.concatenate((g_data, ex_data), axis = 1)
train_y = np.array(train.loc[:, ["Reorg_g", "Reorg_ex"]])

print(train_x.shape)
print(train_y.shape)